In [50]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(precision=3, suppress=True, floatmode='fixed') # Con este comando seteamos el formato de impresión
pd.set_eng_float_format(5)

## Importación Dataset

In [51]:
df = pd.read_csv('properatti.csv', index_col = 0) ### Dataset Completo
## DataFrame Partido en distintas partes relevantes
df_posicion_geografica = df.loc[:,['place_name','state_name','country_name','place_with_parent_names' \
                                   ,'geonames_id','lat','lon','lat-lon', \
                                   'describe','title']] ### Parte con Posición Geográfica
df_localidad=df.loc[:,['place_name','place_with_parent_names','describe','title']] ## Parte con Localidades
df_ambientes = df.loc[:,['rooms','property_type','description','title']]  ## Parte con Ambientes
df_pisos = df.loc[:,['floor','property_type','description','title']]  ## Parte con Pisos
df_precio = df.loc[:,['price','currency','price_aprox_local_currency','price_aprox_usd',\
                      'surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2',\
                      'price_per_m2','description','title']]  ## Parte con Precios y Superficies

## Funciones

In [52]:
def busqueda_localidad(x):
    pattern = r"[\|]([\w\s\.]*)[\|]([\w\s\.]*)[\|]([\w\s\.]*)[\|]([\w\s\.]*)"
    regex = re.compile(pattern, flags = re.IGNORECASE | re.UNICODE)
    m = regex.search(x)
    if m:
        return m.group(1),m.group(2),m.group(3),m.group(4)
def busqueda_ambientes(x):
    '''
    Busca la palabra amb (de ambientes) y devuelve el número que tiene por delante
    '''
    try:
        pattern = '(\d+)\s*amb'
        regex = re.compile(pattern, flags = re.IGNORECASE | re.UNICODE)
        m = regex.search(x)
        if m:
            return float(m.group(1))
        else:
            return np.NaN
    except:
        return np.NaN
def busqueda_monoambientes(x):
    '''
    Busca en la palabra monoambiente y devuelve 1
    '''
    if type(x) is str:
        if x.lower().find('monoambiente') >-1:
            return 1
def busqueda_moneda(x):
    try:
        pattern = 'U\$D\s*([\d\.]+)'
        regex = re.compile(pattern, flags = re.IGNORECASE | re.UNICODE)
        m = regex.search(x)
        if m:
            return float(m.group(1).replace('.',''))
        else:
            np.NaN
    except:
        return np.NaN

## Breve Descripción del Dataset

In [21]:
df.head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3.43079E+06,"-34.6618237,-58.5088387",-34.66182E+00,-58.50884E+00,...,40.00000E+00,1.12727E+03,1.55000E+03,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3.43204E+06,"-34.9038831,-57.9643295",-34.90388E+00,-57.96433E+00,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3.43079E+06,"-34.6522615,-58.5229825",-34.65226E+00,-58.52298E+00,...,55.00000E+00,1.30909E+03,1.30909E+03,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3.43133E+06,"-34.6477969,-58.5164244",-34.64780E+00,-58.51642E+00,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3.43555E+06,"-38.0026256,-57.5494468",-38.00263E+00,-57.54945E+00,...,35.00000E+00,1.82857E+03,1.82857E+03,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [22]:
df.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,102.50300E+03,69.67000E+03,69.67000E+03,100.81000E+03,100.81000E+03,100.81000E+03,81.89200E+03,101.31300E+03,68.61700E+03,87.65800E+03,7.89900E+03,47.39000E+03,14.26200E+03
mean,3.57444E+06,-34.62621E+00,-59.26629E+00,468.52593E+03,4.22940E+06,239.70058E+03,233.79533E+00,133.05018E+00,2.16009E+03,6.91222E+03,17.45234E+00,3.08084E+00,5.00923E+03
std,354.13062E+03,1.98094E+00,2.29992E+00,2.26010E+06,6.90471E+06,391.32388E+03,1.78222E+03,724.35148E+00,2.75929E+03,28.37864E+03,120.24362E+00,1.86077E+00,120.44026E+03
min,3.42721E+06,-54.82398E+00,-75.67893E+00,0.00000E+00,0.00000E+00,0.00000E+00,0.00000E+00,0.00000E+00,600.00000E-03,1.51020E+00,1.00000E+00,1.00000E+00,1.00000E+00
25%,3.43023E+06,-34.66906E+00,-58.72704E+00,110.00000E+03,1.58331E+06,89.73388E+03,50.00000E+00,45.00000E+00,1.21818E+03,1.55000E+03,1.00000E+00,2.00000E+00,1.00000E+03
50%,3.43391E+06,-34.59799E+00,-58.48013E+00,185.00000E+03,2.55845E+06,145.00000E+03,84.00000E+00,75.00000E+00,1.80000E+03,2.21311E+03,3.00000E+00,3.00000E+00,2.00000E+03
75%,3.83667E+06,-34.44130E+00,-58.39591E+00,420.00000E+03,4.67579E+06,265.00000E+03,200.00000E+00,150.00000E+00,2.48641E+03,3.35555E+03,6.00000E+00,4.00000E+00,4.00000E+03
max,6.94890E+06,4.54584E+00,-53.73333E+00,650.00000E+06,821.27110E+06,46.54544E+06,200.00000E+03,187.00000E+03,206.33333E+03,4.00000E+06,3.15000E+03,32.00000E+00,10.00150E+06


In [23]:
df.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121220 entries, 0 to 121219
Data columns (total 25 columns):
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 non-null float64
price_usd_per_m2              68617 n

In [15]:
df.isnull().sum()

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

In [17]:
df[['price','currency']].loc[(~df.price.isnull()) & (df.currency.isnull())] 
##la fila que tiene el precio pero no tiene currency es la 9761, El precio esta en 0.


,price,currency
9761,0.00000E+00,NaN


# Parte Ambientes

In [53]:
df_ambientes_nulos=df_ambientes[df_ambientes.rooms.isnull()] ## Nos quedamos solo con los elementos nulos

In [54]:
df_ambientes_nulos.rooms.fillna(df_ambientes_nulos.description.apply(busqueda_ambientes),inplace=True)
## Rellena los Ambientes vacíos en base a la cantidad de ambientes dichas en la descripción

In [48]:
df_ambientes_nulos.rooms.fillna(df_ambientes_nulos.description.apply(busqueda_monomambientes),inplace=True)
## Rellena los Ambientes vacíos en base a si es un monoambiente o no (presente en la descripción)

In [49]:
df_ambientes_nulos ## Estado de los Ambientes al presente

,rooms,property_type,description,title
0,2.00000E+00,PH,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,NaN,apartment,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,2.00000E+00,apartment,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,3.00000E+00,PH,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,NaN,apartment,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...
5,NaN,house,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ..."
6,NaN,PH,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,..."
7,1.00000E+00,apartment,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...
8,NaN,apartment,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,..."
9,NaN,house,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO


## A Ordenar

In [18]:
df_posicion_geografica.head(5)

,place_name,state_name,country_name,place_with_parent_names,geonames_id,lat,lon,lat-lon,describe,title
0,Mataderos,Capital Federal,Argentina,|Argentina|Capital Federal|Mataderos|,3.43079E+06,-34.66182E+00,-58.50884E+00,"-34.6618237,-58.5088387",NaN,2 AMB TIPO CASA SIN EXPENSAS EN PB
1,La Plata,Bs.As. G.B.A. Zona Sur,Argentina,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,3.43204E+06,-34.90388E+00,-57.96433E+00,"-34.9038831,-57.9643295",NaN,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,Mataderos,Capital Federal,Argentina,|Argentina|Capital Federal|Mataderos|,3.43079E+06,-34.65226E+00,-58.52298E+00,"-34.6522615,-58.5229825",NaN,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,Liniers,Capital Federal,Argentina,|Argentina|Capital Federal|Liniers|,3.43133E+06,-34.64780E+00,-58.51642E+00,"-34.6477969,-58.5164244",NaN,PH 3 amb. cfte. reciclado
4,Centro,Buenos Aires Costa Atlántica,Argentina,|Argentina|Buenos Aires Costa Atlántica|Mar de...,3.43555E+06,-38.00263E+00,-57.54945E+00,"-38.0026256,-57.5494468",NaN,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


In [247]:
df_posicion_geografica.loc[df.place_name.isnull()] ## Localidades que están vacías.

,place_name,state_name,country_name,place_with_parent_names,geonames_id,lat,lon,lat-lon
6489,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.40090E+00,-58.63810E+00,"-34.4008968545,-58.63809847"
10201,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.40070E+00,-58.63807E+00,"-34.400704,-58.638067"
11451,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.40269E+00,-58.62626E+00,"-34.4026880384,-58.6262613379"
14839,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.40080E+00,-58.63817E+00,"-34.4007994779,-58.6381735719"
18622,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.40328E+00,-58.62635E+00,"-34.4032781195,-58.6263503945"
21922,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.39970E+00,-58.63841E+00,"-34.399704,-58.638405"
23664,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.40052E+00,-58.64180E+00,"-34.4005161998,-58.6417999185"
24722,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.40073E+00,-58.63778E+00,"-34.4007292,-58.6377783"
38856,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.42509E+00,-58.57966E+00,"-34.425087,-58.5796585"
45970,NaN,Bs.As. G.B.A. Zona Norte,Argentina,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,NaN,-34.39607E+00,-58.63738E+00,"-34.396074,-58.63738"


In [218]:
df_localidad.iloc[4:5,1].apply(busqueda_localidad)


4    (Argentina, Buenos Aires Costa Atlántica, Mar ...
Name: place_with_parent_names, dtype: object

In [271]:
df_precio_nulo = df[df.price.isnull()]

In [219]:
df_localidad.place_with_parent_names.apply(busqueda_localidad)

0                 (Argentina, Capital Federal, Mataderos, )
1           (Argentina, Bs.As. G.B.A. Zona Sur, La Plata, )
2                 (Argentina, Capital Federal, Mataderos, )
3                   (Argentina, Capital Federal, Liniers, )
4         (Argentina, Buenos Aires Costa Atlántica, Mar ...
5                   (Argentina, Entre Ríos, Gualeguaychú, )
6         (Argentina, Bs.As. G.B.A. Zona Norte, Vicente ...
7                  (Argentina, Capital Federal, Belgrano, )
8                  (Argentina, Capital Federal, Belgrano, )
9                          (Argentina, Santa Fe, Rosario, )
10                          (Argentina, Córdoba, Córdoba, )
11        (Argentina, Bs.As. G.B.A. Zona Norte, San Migu...
12        (Argentina, Bs.As. G.B.A. Zona Norte, San Isid...
13        (Argentina, Capital Federal, Palermo, Palermo ...
14        (Argentina, Capital Federal, Palermo, Palermo ...
15        (Argentina, Bs.As. G.B.A. Zona Norte, San Migu...
16                (Argentina, Capital Fe

In [220]:
df_localidad.place_with_parent_names.head(20)

0                 |Argentina|Capital Federal|Mataderos|
1           |Argentina|Bs.As. G.B.A. Zona Sur|La Plata|
2                 |Argentina|Capital Federal|Mataderos|
3                   |Argentina|Capital Federal|Liniers|
4     |Argentina|Buenos Aires Costa Atlántica|Mar de...
5                   |Argentina|Entre Ríos|Gualeguaychú|
6     |Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...
7                  |Argentina|Capital Federal|Belgrano|
8                  |Argentina|Capital Federal|Belgrano|
9                          |Argentina|Santa Fe|Rosario|
10                          |Argentina|Córdoba|Córdoba|
11      |Argentina|Bs.As. G.B.A. Zona Norte|San Miguel|
12    |Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...
13     |Argentina|Capital Federal|Palermo|Palermo Soho|
14     |Argentina|Capital Federal|Palermo|Palermo Soho|
15    |Argentina|Bs.As. G.B.A. Zona Norte|San Miguel...
16                |Argentina|Capital Federal|Mataderos|
17    |Argentina|Bs.As. G.B.A. Zona Norte|Tigre|

In [221]:
df.iloc[2]

operation                                                                  sell
property_type                                                         apartment
place_name                                                            Mataderos
place_with_parent_names                   |Argentina|Capital Federal|Mataderos|
country_name                                                          Argentina
state_name                                                      Capital Federal
geonames_id                                                         3.43079E+06
lat-lon                                                 -34.6522615,-58.5229825
lat                                                               -34.65226E+00
lon                                                               -58.52298E+00
price                                                              72.00000E+03
currency                                                                    USD
price_aprox_local_currency              

## Otros

In [56]:
#Función caduca, no usar.
def update(x,y,nullvalue = None):
    x = np.where(x==nullvalue, y, x)
    #x = np.where(x.eq(0), y, x[])
    return x
update(df_ambientes_nulos.loc[:,'rooms'],df_ambientes_nulos.description.apply(busqueda_monomambientes),np.NaN)

array([2.000,   nan, 2.000, ...,   nan, 2.000,   nan])

In [55]:
## Para los precios
df_precio_nulo.title.apply(busqueda_moneda)[~df_precio_nulo.title.apply(busqueda_moneda).isnull()].sort_values(ascending=False)

NameError: name 'df_precio_nulo' is not defined